In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'fourier'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_fourier_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1020.30,0.03,-0.07,-0.01,0.03,0.00,-0.00,-0.01,0.00,-0.02,...,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.0,-0.00,-0.00
3,0.03,1306.59,0.01,0.00,0.05,0.02,0.01,0.02,0.01,0.01,...,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.0,-0.00,0.00
4,-0.07,0.01,1131.26,-0.02,-0.02,-0.00,0.03,-0.01,0.01,0.01,...,-0.01,0.00,0.00,-0.00,0.00,-0.00,0.00,0.0,0.00,-0.00
5,-0.01,0.00,-0.02,612.90,-0.02,0.01,-0.02,-0.00,-0.01,-0.01,...,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.0,0.00,0.00
6,0.03,0.05,-0.02,-0.02,434.06,-0.01,-0.00,-0.01,-0.01,-0.01,...,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.0,-0.00,-0.00
7,0.00,0.02,-0.00,0.01,-0.01,277.95,-0.00,0.00,0.01,0.00,...,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.0,-0.00,-0.00
8,-0.00,0.01,0.03,-0.02,-0.00,-0.00,207.91,-0.00,0.00,0.01,...,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.0,-0.00,-0.00
9,-0.01,0.02,-0.01,-0.00,-0.01,0.00,-0.00,187.78,-0.00,0.01,...,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.0,0.00,-0.00
10,0.00,0.01,0.01,-0.01,-0.01,0.01,0.00,-0.00,135.48,-0.00,...,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.0,-0.00,0.00
11,-0.02,0.01,0.01,-0.01,-0.01,0.00,0.01,0.01,-0.00,119.43,...,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.0,-0.00,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,0.00003,-0.00006,-0.00002,0.00004,0.00001,-0.00001,-0.00003,0.00001,-0.00005,...,-0.00001,-0.00001,-0.00007,0.00007,0.00003,0.00000,-0.00001,-0.00006,-0.00001,-0.00001
3,0.00003,1.00000,0.00001,0.00000,0.00006,0.00004,0.00002,0.00004,0.00003,0.00001,...,-0.00004,-0.00000,-0.00003,-0.00001,0.00005,0.00002,-0.00001,0.00002,-0.00001,0.00003
4,-0.00006,0.00001,1.00000,-0.00002,-0.00003,-0.00000,0.00006,-0.00001,0.00002,0.00003,...,-0.00007,0.00002,0.00005,-0.00002,0.00000,-0.00007,0.00002,0.00006,0.00002,-0.00001
5,-0.00002,0.00000,-0.00002,1.00000,-0.00003,0.00001,-0.00004,-0.00001,-0.00003,-0.00005,...,0.00003,0.00002,0.00003,0.00000,-0.00001,-0.00002,-0.00001,-0.00002,0.00011,0.00002
6,0.00004,0.00006,-0.00003,-0.00003,1.00000,-0.00002,-0.00001,-0.00003,-0.00003,-0.00002,...,-0.00002,0.00002,0.00004,-0.00004,0.00002,-0.00000,0.00003,-0.00003,-0.00003,-0.00002
7,0.00001,0.00004,-0.00000,0.00001,-0.00002,1.00000,-0.00001,0.00001,0.00004,0.00002,...,0.00001,0.00006,-0.00001,0.00005,-0.00005,0.00002,0.00001,0.00003,-0.00003,-0.00001
8,-0.00001,0.00002,0.00006,-0.00004,-0.00001,-0.00001,1.00000,-0.00002,0.00002,0.00004,...,-0.00002,-0.00004,-0.00000,0.00000,-0.00001,-0.00002,0.00003,-0.00002,-0.00002,-0.00005
9,-0.00003,0.00004,-0.00001,-0.00001,-0.00003,0.00001,-0.00002,1.00000,-0.00001,0.00003,...,0.00003,-0.00001,-0.00004,-0.00001,-0.00001,0.00001,-0.00000,0.00005,0.00003,-0.00001
10,0.00001,0.00003,0.00002,-0.00003,-0.00003,0.00004,0.00002,-0.00001,1.00000,-0.00003,...,0.00002,-0.00000,0.00003,-0.00006,-0.00001,0.00005,0.00002,-0.00003,-0.00005,0.00003
11,-0.00005,0.00001,0.00003,-0.00005,-0.00002,0.00002,0.00004,0.00003,-0.00003,1.00000,...,0.00001,-0.00002,-0.00002,0.00003,-0.00001,0.00003,-0.00002,0.00001,-0.00003,-0.00003


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.18447363510073617

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.36043060e+04 4.00803913e+04 3.22920442e+04 1.17087980e+04
 5.86347146e+03 2.40829444e+03 1.35325430e+03 1.09902476e+03
 9.12186800e+02 5.68858387e+02 4.36855493e+02 2.35957354e+02
 1.89275352e+02 7.90189620e+01 5.20420671e+01 4.66215505e+01
 2.80312157e+01 2.02331863e+01 1.40734284e+01 9.05220331e+00
 6.82893193e+00 5.37453473e+00 2.78672578e+00 1.22848442e+00
 4.70647747e-01 3.20835955e-01 6.50717190e-03 4.12989276e-03
 5.07359320e-04 1.89478940e-04 3.93626031e-06 1.05713622e-26]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.056603,0.992915,-0.102148,-0.016442,-0.010115,-0.006033,-0.004429,-0.003965,-0.002836,-0.002498,...,-0.000208,-0.000451,-0.000084,-0.000074,-0.000009,-0.000008,-0.000003,-0.000002,-3.214483e-07,-6.868177e-08
1,-0.164478,0.090967,0.981730,-0.022903,-0.013355,-0.007717,-0.005575,-0.005034,-0.003574,-0.003131,...,-0.000264,-0.000561,-0.000100,-0.000094,-0.000012,-0.000012,-0.000003,-0.000002,-2.913187e-07,-2.044218e-07
2,0.983921,0.071182,0.156823,-0.037781,-0.020520,-0.011585,-0.008357,-0.007504,-0.005303,-0.004679,...,-0.000383,-0.000834,-0.000157,-0.000133,-0.000017,-0.000016,-0.000005,-0.000004,-5.906565e-07,-2.961228e-07
3,0.031064,0.020045,0.025480,0.997156,-0.051355,-0.020041,-0.013442,-0.011824,-0.008123,-0.007089,...,-0.000557,-0.001229,-0.000223,-0.000201,-0.000027,-0.000024,-0.000008,-0.000006,-2.758590e-07,-3.460193e-07
4,0.018055,0.013009,0.015743,0.048277,0.996809,-0.040583,-0.022888,-0.019571,-0.012669,-0.010892,...,-0.000832,-0.001816,-0.000329,-0.000301,-0.000038,-0.000035,-0.000011,-0.000009,-1.279886e-06,-6.764268e-07
5,0.009818,0.007424,0.008739,0.018925,0.035530,0.994930,-0.061570,-0.044312,-0.022336,-0.018380,...,-0.001229,-0.002695,-0.000495,-0.000437,-0.000060,-0.000051,-0.000017,-0.000012,-1.929304e-06,-9.526135e-07
6,0.006134,0.004707,0.005459,0.011062,0.017799,0.047319,0.979347,-0.168758,-0.034413,-0.025839,...,-0.001421,-0.003146,-0.000568,-0.000507,-0.000068,-0.000061,-0.000019,-0.000015,-2.173990e-06,-1.334538e-06
7,0.006183,0.004733,0.005532,0.010928,0.017135,0.039126,0.138171,0.957204,-0.052490,-0.036611,...,-0.001760,-0.003926,-0.000714,-0.000635,-0.000085,-0.000074,-0.000025,-0.000017,-2.255902e-06,-1.448167e-06
8,0.007922,0.006124,0.007111,0.013898,0.021363,0.044752,0.109904,0.212015,-0.107442,-0.066996,...,-0.002757,-0.006117,-0.001110,-0.000989,-0.000129,-0.000117,-0.000038,-0.000026,-3.899363e-06,-1.827689e-06
9,0.005165,0.004004,0.004647,0.008899,0.013217,0.024154,0.042140,0.055348,0.974998,-0.184102,...,-0.002914,-0.006544,-0.001164,-0.001056,-0.000138,-0.000120,-0.000040,-0.000030,-4.585331e-06,-1.852083e-06


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0566032489976813,
 0.9090330573058893,
 0.8431773258574845,
 0.002844069813893846,
 0.003191406363617033,
 0.005069813768808551,
 0.020652773374305156,
 0.04279586502973998,
 1.107441951926659,
 1.1841024072874962,
 1.0637357999829158,
 1.0350577078701917,
 1.0431576322443177,
 1.1021278828094343,
 1.0144150892391757,
 1.0926057006512058,
 1.0364654460298837,
 1.0805933340603908,
 1.0813870595793484,
 0.9942614037409322,
 0.9921832416925753,
 1.0845206015613065,
 0.042142732800395266,
 0.963175061738689,
 0.12037266545671588,
 0.12600130351111372,
 0.12815253158652384,
 0.13493572382720664,
 0.08005503400277514,
 0.0829632963480249,
 0.11637497233949279,
 0.11561542407514735]